In [28]:
import requests
from bs4 import BeautifulSoup
import json


In [29]:
from currency_converter import CurrencyConverter, SINGLE_DAY_ECB_URL
cc = CurrencyConverter(SINGLE_DAY_ECB_URL)

In [30]:
cc.convert(650, 'NOK')

60.671115881831334

In [51]:
from_to = "Ulaanbaatar/Cancun"

In [52]:
base_url = "https://www.rome2rio.com/map/"
    
tmp_url = base_url + from_to
    
print(tmp_url)
    

https://www.rome2rio.com/map/Ulaanbaatar/Cancun


In [53]:
r = requests.get(tmp_url)
print(r.status_code)

200


In [ ]:
soup = BeautifulSoup(r.text, "html.parser")
print(soup.prettify())

In [55]:
# finds common info about the trip
title = soup.find('meta', property="og:title")
print(title['content'])

Ulaanbaatar to Cancún - 2 ways to travel via plane, and Air taxi


In [56]:
# gets quickest and cheapest ways
description = soup.find('meta', property="og:description")
print(description['content'])

The cheapest way to get from Ulaanbaatar to Cancún costs only $28792, and the quickest way takes just 22 hours. Find the travel option that best suits you.


In [57]:
# finds nodes geolocation info
nodes = soup.find('meta', id="deeplinkNodes")
nodes['content']

'[["GeoCoderResultV1","Ulaanbaatar","Ulaanbaatar, Mongolia",47.90771,106.8832,"Capital","MN","Ulaanbaatar","False",2.5],["GeoCoderResultV1","Cancún","Cancún, Mexico",21.17429,-86.84656,"City","MX","Cancún","False",2.5]]'

In [58]:
# find trip`s pathes
pathes = soup.find('meta', id="deeplinkTrip")
pathes = json.loads(pathes['content'])[2][1]

In [59]:
# calculates number of pathes
num_pathes = len([i for i in pathes])
print(f'{num_pathes = }')

num_pathes = 2


In [ ]:
# get all pathes wirh numbers
for i, item in enumerate(pathes, start=1):
    print('path ', i, '\n\t', item, '\n\n')

In [ ]:
# unpacking all routes from each path
for path_id, path in enumerate(pathes, start=1):
    for i, route in enumerate(path):
        print(path_id, '\n\t', i, '\t', route, '\n')

In [ ]:
# unpacking routes info from the section 8 (except last one [:-1], because it is 'hotel')
for path_id, path in enumerate(pathes, start=1):
    for i, route in enumerate(path[8][:-1]):
        print(path_id, '\n\t', i, '\t', route, '\n')

    

In [63]:
# unpacking each route's info from the section 8 (except last one [:-1], because it is 'hotel')
for path_id, path in enumerate(pathes):
    print(f'path # {path_id}\n')
    for i, route in enumerate(path[8][:-1]):
        for j, way in enumerate(route):
            print(path_id, '\t', i, '\t', j, '\t', way, '\n')


path # 0

0 	 0 	 0 	 car 

0 	 0 	 1 	 taxi 

0 	 0 	 2 	 yellow 

0 	 0 	 3 	 2630.555 

0 	 0 	 4 	 0 

0 	 0 	 5 	 46.03791 

0 	 0 	 6 	 ['node', 'Ulaanbaatar', 47.90771, 106.8832, 'MN', 'Asia/Ulaanbaatar', 2.5] 

0 	 0 	 7 	 ['airport', 'Ulaanbaatar (UBN)', 47.65158, 106.8218] 

0 	 0 	 8 	  

0 	 0 	 9 	 False 

0 	 0 	 10 	  

0 	 0 	 11 	  

0 	 0 	 12 	 False 

0 	 0 	 13 	 [[280, 'MXN', 286.65, 'False', ''], [340, 'MXN', 338.77, 'False', ''], [340, 'MXN', 338.77, 'False', '']] 

0 	 0 	 14 	 [[50000, 'MNT', 51741.72, 'False', ''], [60000, 'MNT', 61149.3, 'False', ''], [65000, 'MNT', 61149.3, 'False', '']] 

0 	 0 	 15 	 53 

0 	 0 	 16 	 53 

0 	 0 	 17 	 [['Ulaanbaatar Taxi', '1900-1991', 'https://www.facebook.com/Ulaanbaatar.Taxi/', 'facebook.com']] 

0 	 0 	 18 	 [] 

0 	 0 	 19 	 [] 

0 	 0 	 20 	 [] 

0 	 0 	 21 	  

0 	 0 	 22 	  

0 	 0 	 23 	 [] 

0 	 1 	 0 	 flight 

0 	 1 	 1 	 turquoise 

0 	 1 	 2 	 ['UBN', 'Ulaanbaatar', '', 47.65158, 106.8218, 'Töv', 'Mongolia'

In [44]:
# unpacking info from the section 9 - local moving inside the city, from city center to airport etc.
for path_id, path in enumerate(pathes, start=1):
    for i, route in enumerate(path[9]):
        for j, way in enumerate(route[2]):
            print(path_id, '\n\t', i, '\t', j, '\t', way, '\n')

1 
	 0 	 0 	 0 

1 
	 0 	 1 	 ['station', 'Cancun Airport', 21.03995, -86.8745, '', '', -1] 

1 
	 0 	 2 	 ['dynamicTransitStation', 'Cancún', 21.17429, -86.84656, '', '', -1] 

1 
	 0 	 3 	  

1 
	 0 	 4 	  

1 
	 0 	 5 	 1253.363 

1 
	 0 	 6 	 1253.363 

1 
	 0 	 7 	 0 

1 
	 0 	 8 	 [['transit', 'shuttle', 'red', 1253.363, 1800, 20.79084, ['station', 'Cancun Airport', 21.03995, -86.8745, 'MX', 'America/Cancun'], ['dynamicTransitStation', 'Cancún', 21.17429, -86.84656, 'MX', 'America/Cancun'], '', 'True', [1253.363, 20.79084, ['station', 'Cancun Airport', 21.03995, -86.8745, 'MX', 'America/Cancun'], ['dynamicTransitStation', 'Cancún', 21.17429, -86.84656, 'MX', 'America/Cancun'], 'shuttle', ['OnDemand'], '', 168, [['Jayride Shuttles', '//static.r2r.io/logos/Trains/96/_JAYRIDE_SHAREDSHUTTLE_MEXICO201904301614_201904301619.png', 'https://www.jayride.com/', 'jayride.com', 1253.363, 20.79084, -1, '', [['https://www.jayride.com/', 'jayride.com', 'Schedules at', 'Jayride Shuttles', '', ''

In [45]:
# unpacking airports', stations' and nodes' coordinates for each route from pathes from the section 10
for path_id, path in enumerate(pathes, start=1):
    for i, route in enumerate(path[10]):
        print(path_id, '\n\t', i, '\t', route, '\n')


1 
	 0 	 ['node', 'Gloucestershire', 51.83333, -2.166667, 'GB', 'Europe/London', 2.5] 

1 
	 1 	 ['station', 'Gloucester, Station Road', 51.86502, -2.24132, 'GB', 'Europe/London'] 

1 
	 2 	 ['station', 'Gloucester', 51.86556, -2.23849, 'GB', 'Europe/London'] 

1 
	 3 	 ['station', 'Bristol Temple Meads', 51.44914, -2.58132, 'GB', 'Europe/London'] 

1 
	 4 	 ['station', 'Bristol Temple Meads', 51.44884, -2.58407, 'GB', 'Europe/London'] 

1 
	 5 	 ['station', 'Bristol Airport', 51.38673, -2.710848, 'GB', 'Europe/London'] 

1 
	 6 	 ['airport', 'Bristol (BRS)', 51.38665, -2.71165] 

1 
	 7 	 ['airport', 'Cancun (CUN)', 21.03995, -86.8745] 

1 
	 8 	 ['station', 'Cancun Airport', 21.03971, -86.87302, 'MX', 'America/Cancun'] 

1 
	 9 	 ['node', 'Cancún', 21.17429, -86.84656, 'MX', 'America/Cancun', 2.5] 

2 
	 0 	 ['node', 'Gloucestershire', 51.83333, -2.166667, 'GB', 'Europe/London', 2.5] 

2 
	 1 	 ['station', 'Gloucester, Station Road', 51.86502, -2.24132, 'GB', 'Europe/London'] 

2 
	 

In [46]:
# unpacking accomodation info for each route from pathes from the section 15
for path_id, path in enumerate(pathes, start=1):
    for i, route in enumerate(path[15]):
        print(path_id, '\n\t', i, '\t', route, '\n')

1 
	 0 	 [['Local', '', '', 'False', 'Schedules', [5, 8, [], 40, [], 'False', -1, -1, -1, 'https://help.rome2rio.com/en/support/solutions/articles/22000239379-are-there-any-age-discounts-for-my-tickets-', 'False', 'False', [], 'False'], 'True', 19], ['OmioAtoc', 'Book at {0}', 'Omio', 'False', 'Schedules and Book', [23, 8, [[40, 'Adult', 'Adults', ''], [10, 'Child', 'Children', '']], 40, [], 'True', -1, -1, 366, 'https://help.rome2rio.com/en/support/solutions/articles/22000239379-are-there-any-age-discounts-for-my-tickets-', 'False', 'False', [], 'False'], 'False', 2]] 

1 
	 1 	 [[[[0, [[0, [0]]], [-1, []], [-1, []], '']]], [[[2, [[0, [1, 0]], [1, [1, 0]]], [-1, []], [-1, []], ['/redirects/omio/?oDateTime={oDateTime}&iDateTime={iDateTime}&adults={adults}&seniors={seniors}&youths={youths}&ages={ages}&checkoutExitLabel={checkoutExitLabel}&originId=319143&destinationId=318541&mode=train&lang=en&currency=MXN&requestId=222-20230121-230342-7993636&agencyIds=_GREAT_WESTERN_RAILWAY', 'Omio', 

In [47]:
# unpacking the prices (min, mid, max) of the path from the section 20
for path_id, path in enumerate(pathes, start=1):
    print(path_id, '\n\t', path[20], '\n')

1 
	 [[7560, 'MXN', 7560.36, 'False', ''], [15447, 'MXN', 15447.47, 'False', ''], [26304, 'MXN', 26304.44, 'False', '']] 

2 
	 [[7872, 'MXN', 7872.36, 'False', ''], [13829, 'MXN', 13829.47, 'False', ''], [21416, 'MXN', 21416.44, 'False', '']] 

3 
	 [[5852, 'MXN', 5852.36, 'False', ''], [13109, 'MXN', 13109.47, 'False', ''], [30466, 'MXN', 30466.44, 'False', '']] 

4 
	 [[6717, 'MXN', 6717.36, 'False', ''], [8019, 'MXN', 8019.47, 'False', ''], [16331, 'MXN', 16331.44, 'False', '']] 

5 
	 [[8852, 'MXN', 8852.36, 'False', ''], [17159, 'MXN', 17159.47, 'False', ''], [28066, 'MXN', 28066.44, 'False', '']] 

6 
	 [[7982, 'MXN', 7982.36, 'False', ''], [13189, 'MXN', 13189.47, 'False', ''], [21686, 'MXN', 21686.44, 'False', '']] 



In [48]:
# unpacking the prices (min, mid, max) for each route of the path from the section 20
for path_id, path in enumerate(pathes, start=1):
    for i, route in enumerate(path[20]):
        print(path_id, '\n\t', i, '\t', route, '\n')

1 
	 0 	 [7560, 'MXN', 7560.36, 'False', ''] 

1 
	 1 	 [15447, 'MXN', 15447.47, 'False', ''] 

1 
	 2 	 [26304, 'MXN', 26304.44, 'False', ''] 

2 
	 0 	 [7872, 'MXN', 7872.36, 'False', ''] 

2 
	 1 	 [13829, 'MXN', 13829.47, 'False', ''] 

2 
	 2 	 [21416, 'MXN', 21416.44, 'False', ''] 

3 
	 0 	 [5852, 'MXN', 5852.36, 'False', ''] 

3 
	 1 	 [13109, 'MXN', 13109.47, 'False', ''] 

3 
	 2 	 [30466, 'MXN', 30466.44, 'False', ''] 

4 
	 0 	 [6717, 'MXN', 6717.36, 'False', ''] 

4 
	 1 	 [8019, 'MXN', 8019.47, 'False', ''] 

4 
	 2 	 [16331, 'MXN', 16331.44, 'False', ''] 

5 
	 0 	 [8852, 'MXN', 8852.36, 'False', ''] 

5 
	 1 	 [17159, 'MXN', 17159.47, 'False', ''] 

5 
	 2 	 [28066, 'MXN', 28066.44, 'False', ''] 

6 
	 0 	 [7982, 'MXN', 7982.36, 'False', ''] 

6 
	 1 	 [13189, 'MXN', 13189.47, 'False', ''] 

6 
	 2 	 [21686, 'MXN', 21686.44, 'False', ''] 

